In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sb
import pickle

In [ ]:
cross_df = pd.read_csv('oasis_cross-sectional.csv')

In [ ]:
long_df = pd.read_csv("oasis_longitudinal.csv")

In [ ]:
long_df.head(5)

In [ ]:
cross_df.head(5)

In [ ]:
a = long_df.iloc[:,1]

In [ ]:
b = cross_df[cross_df['ID'].isin([a])]
b
#Cross sectional data and longitudinal data are not related

## Longitudinal data set into consideration

In [ ]:
long_df = long_df.drop("Subject ID",axis = 1)
long_df = long_df.drop("MRI ID",axis =1)
#dropping Hand column
long_df=long_df.drop("Hand",axis=1)
import matplotlib.pyplot as plt

In [ ]:
#removing NaN values
long_df.SES = long_df.SES.fillna(round(long_df.SES.mean()))

In [ ]:
long_df.MMSE = long_df.MMSE.fillna(round(long_df.MMSE.mean()))
long_df['Group'] = long_df['Group'].replace('Converted','Demented') 
#creating dummy variables
long_df = pd.get_dummies(data= long_df,columns = {'Group','M/F'})
long_df = long_df.rename(columns={'M/F_F':'Female','M/F_M':'Male','Group_Demented':'Demented','Group_Nondemented':'Non-Demented'})
#Male=1 and Female=0
long_df=long_df.drop("Female",axis=1)
long_df = long_df.rename(columns={"Male":"Gender"})
#Demented=1 and Non-demented=0
long_df = long_df.drop("Non-Demented",axis=1)
long_df = long_df.rename(columns={"Demented":"Group"})

## Exploratory Data Analysis

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 8,10

In [ ]:
def bar_graph(X):
    demented = long_df[long_df['Group']==1][X].value_counts()
    non_demented = long_df[long_df['Group']==0][X].value_counts()
    bar_df = pd.DataFrame([demented,non_demented])
    bar_df =bar_df.rename(columns={0:'Female',1:'Male'})
    bar_df.index = ['Demented','Non-demented']
    bar_df.plot(kind="bar",stacked="true")
    

In [ ]:
bar_graph('Gender')
plt.title("Gender vs Dementia")
plt.ylim(0,200,10)
plt.ylabel('Number of people')
plt.legend(loc="center")
plt.show()

In [ ]:
rcParams['figure.figsize'] = 10,10

In [ ]:
def box_graph(X):
    demented = long_df[long_df['Group']==1][X]
    non_demented = long_df[long_df['Group']==0][X]
    box_data = [demented,non_demented]
    plt.boxplot(box_data,patch_artist=True,labels=['Demented','Non-demented'],widths = (0.5,0.5),vert=False,showfliers=False)

In [ ]:
box_graph('MMSE')
plt.xlim(12,36,2)
plt.xlabel('MMSE')
plt.title('Impact of MMSE')
plt.show()

In [ ]:
def violin_graph(X):
    w=long_df['Group'].replace([0,1],['Non-demented','Demented'])
    y=long_df[X] 
    violin_df=pd.DataFrame([w,y]).T
    sb.violinplot(x=y, y=w,data=violin_df,palette='rainbow',linewidth=3, orient='h')    
      

In [ ]:
violin_graph('EDUC')
plt.xlabel("Education(in years)",fontsize=14)
plt.ylabel("Group",fontsize = 14)
plt.xticks(np.arange(2,28,2))
plt.title("Impact of Education")
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(long_df) 
X = scaler.transform(long_df);X

In [ ]:
y = long_df['Group']
x_pca = long_df[['Age', 'EDUC', 'SES', 'MMSE', 'eTIV',
       'nWBV', 'ASF', 'Gender']]

In [ ]:
#applying PCA to training features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(x_pca) 
X = scaler.transform(x_pca);X

In [ ]:
#plotting of top two components of pca to explain linear separability of two components through the plotting
from sklearn.decomposition import PCA
covar_matrix = PCA(n_components = 2)
yoyo= covar_matrix.fit_transform(X)
variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
sb.scatterplot(x=yoyo[:,0],y=yoyo[:,1],hue=y)
var=np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3)*100)
var #cumulative sum of variance explained with [n] features

In [ ]:
plt.ylabel('% Variance Explained')
plt.xlabel('# of Features')
plt.title('PCA Analysis')
plt.ylim(30,100.5)
plt.style.context('seaborn-whitegrid')
plt.plot(var)

## Splitting Data into train and test sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
long_df.columns

In [ ]:
columns = ['Age', 'EDUC', 'SES', 'MMSE', 'eTIV',
       'nWBV', 'ASF', 'Gender']

In [ ]:
def split_data():
    y = long_df['Group']
    x = long_df[['Age', 'EDUC', 'SES', 'MMSE', 'eTIV',
       'nWBV', 'ASF', 'Gender']]
    X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.30,random_state = 0)
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,Y_train,Y_test

In [ ]:
X_train,X_test,Y_train,Y_test = split_data()

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, roc_curve, auc,precision_score
compare=[]

In [ ]:
    LogModel = LogisticRegression(solver='lbfgs').fit(X_train,Y_train)#vanilla logistic regression
    Y_predicted = LogModel.predict(X_test)
    acc = accuracy_score(Y_test,Y_predicted)
    recall = recall_score(Y_test,Y_predicted,pos_label=1)
    fpr, tpr, thresholds = roc_curve(Y_test,Y_predicted,pos_label=1)
    AUC = auc(fpr,tpr)
    compare.append(['Logistic Regression',acc,recall,AUC,fpr,tpr,thresholds])
    compare
#Explain the coefficients and results
#explain vs predict

### CDR(Clinical dementia rating)

CDR is solely responsible to say about the rating of dementia. Hence do not take the feature into account.

In [ ]:
LogModel.intercept_

In [ ]:
#add the regression coefficients(how much impact the outputs have on the features)
LogModel.coef_.T

In [ ]:
np.array([columns]).T

In [ ]:
feature_importance=pd.DataFrame(np.hstack((np.array([columns]).T, LogModel.coef_.T)), 
                                columns=['feature', 'importance'])
feature_importance['importance']=pd.to_numeric(feature_importance['importance'])
feature_importance.sort_values(by='importance', ascending=False)

## Interpretation of reg coefficients(in terms of odd ratios)

Keeping everything else constant, The odds of men getting dementia as compared to women is exp(0.716563)=2.04
times more.

All else being constant, if a person gets older by a year, the probability that the person will get dementia 
increases by 1/(1+exp(-(-0.452391+0.62704977))= 54.36%


In [ ]:
#confusion matrix
cnf_matrix = confusion_matrix(Y_test, Y_predicted)
df_cm = pd.DataFrame(cnf_matrix,columns={'Non-demented','Demented'},index={'Non-demented','Demented'})
sb.heatmap(df_cm,annot=True,cbar=False)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')

## Random Forest


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RFModel = RandomForestClassifier(n_estimators=15, max_features=6,max_depth=10,random_state=0).fit(X_train,Y_train)
Y_predicted = RFModel.predict(X_test)
acc = accuracy_score(Y_test,Y_predicted)
recall = recall_score(Y_test,Y_predicted,pos_label=1)
fpr,tpr, thresholds = roc_curve(Y_test,Y_predicted,pos_label=1)
AUC=auc(fpr,tpr)
compare.append(['Random Forest',acc,recall,AUC,fpr,tpr,thresholds])

In [ ]:
np.array([RFModel.feature_importances_]).T

In [ ]:
np.array([columns]).T

In [ ]:
feature_importance=pd.DataFrame(np.hstack((np.array([columns]).T, np.array([RFModel.feature_importances_]).T)), 
                                columns=['feature', 'importance'])
feature_importance['importance']=pd.to_numeric(feature_importance['importance'])
feature_importance.sort_values(by='importance', ascending=False)

In [ ]:
#confusion matrix
cnf_matrix = confusion_matrix(Y_test, Y_predicted)
df_cm = pd.DataFrame(cnf_matrix,columns={'Non-demented','Demented'},index={'Non-demented','Demented'})
sb.heatmap(df_cm,annot=True,cbar=False)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')

In [ ]:
compare

In [ ]:
compare = pd.DataFrame(data=compare,columns=['Model', 'Accuracy', 'Recall', 'AUC', 'FPR', 'TPR', 'TH']);compare

## Performing XGBoost

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.datasets import dump_svmlight_file
from sklearn.model_selection import GridSearchCV
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
# use DMatrix for xgbosot
dtrain = xgb.DMatrix(X_train, label=Y_train)
dtest = xgb.DMatrix(X_test, label=Y_test)

# use svmlight file for xgboost
dump_svmlight_file(X_train, Y_train, 'dtrain.svm', zero_based=True)
dump_svmlight_file(X_test, Y_test, 'dtest.svm', zero_based=True)
dtrain_svm = xgb.DMatrix('dtrain.svm')
dtest_svm = xgb.DMatrix('dtest.svm')

# set xgboost params
param = {
    'n_estimators':1100,
     'max_depth': 5,  # the maximum depth of each tree
    'eta': 0.05,  # the training step for each iteration
    'silent': 1, # logging mode - quiet
    'objective': 'binary:logistic' # error evaluation for multiclass training
#     'min_child_weight':1,
#     'gamma':0.4,
#     'subsample':0.9,
#  'colsample_bytree':0.8
    }  
num_round = 15000  # the number of training iterations

In [ ]:
# training and testing - svm file
bst_svm = xgb.train(param, dtrain_svm, num_round)
preds = bst_svm.predict(dtest_svm)

# extracting most confident predictions
best_preds_svm = [round(line) for line in preds]
print("Accuracy",accuracy_score(best_preds_svm,Y_test))
print("Recall",recall_score(best_preds_svm,Y_test))
print("Precision",precision_score(best_preds_svm,Y_test))

In [ ]:
cf = confusion_matrix(Y_test, best_preds_svm)
df_cm = pd.DataFrame(cf,columns=['Non-demented','Demented'],index=['Non-demented','Demented'])
sb.heatmap(df_cm,annot=True,cbar=False)
tn = cf[0][0]
fp = cf[0][1]
fn = cf[1][0]
tp = cf[1][1]
sensitivity= tp/(tp+fn)
specificity = tn/(tn+fp);specificity

In [ ]:
#Creating a scatterplot to distinguish which points are classified correctly and which are not.
#The x marks represents which data points are misrepresented

from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler().fit(x_pca) 

xright,yright = X_test[Y_test==best_preds_svm],Y_test[Y_test==best_preds_svm]
xwrong,ywrong = X_test[Y_test!=best_preds_svm],Y_test[Y_test!=best_preds_svm]

X_r = scaler.transform(xright)
X_w = scaler.transform(xwrong)
yoyo_new_right = covar_matrix.transform(X_r)
yoyo_new_wrong = covar_matrix.transform(X_w)

sb.scatterplot(x=yoyo_new_right[:,0],y=yoyo_new_right[:,1],hue = yright)
sb.scatterplot(x=yoyo_new_wrong[:,0],y=yoyo_new_wrong[:,1],hue = ywrong,marker = "x")

In [ ]:
#Other method
# ##fit model no training data
# model = XGBClassifier(learning_rate =0.05,
#  n_estimators=5000,
#  max_depth=5,
#  min_child_weight=1,
#  gamma=0,
#  subsample=0.8,
#  colsample_bytree=0.8,
#  objective= 'binary:logistic',
#  nthread=15,
#  scale_pos_weight=1,
#  seed=27)
# model.fit(X_train, Y_train)
# # make predictions for test data
# y_pred = model.predict(X_test)
# predictions = [round(value) for value in y_pred]
# # evaluate predictions
# accuracy = accuracy_score(Y_test, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))

### Tuning max_depth and min_child_weight

In [ ]:
param_test1 = {
 'max_depth':range(3,13,2),
 'min_child_weight':range(1,10,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.08, n_estimators=5000, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=15, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='accuracy',n_jobs=4,iid=False, cv=2)
gsearch1.fit(X_train,Y_train)
gsearch1.best_params_, gsearch1.best_score_

### Tuning gamma

In [ ]:
param_test2 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.08, n_estimators=5000, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=15, scale_pos_weight=1, seed=27), 
 param_grid = param_test2, scoring='accuracy',n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_train,Y_train)
gsearch2.best_params_, gsearch1.best_score_

### Tuning subsample and colsample_bytree

In [ ]:
param_test3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.08, n_estimators=5000, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=15, scale_pos_weight=1, seed=27), 
 param_grid = param_test3, scoring='accuracy',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train,Y_train)
gsearch3.best_params_, gsearch1.best_score_

### Tuning n_estimators

In [ ]:
param_test4 = {
 'n_estimators':[i for i in range(100,10000,500)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.08, n_estimators=5000, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=15, scale_pos_weight=1, seed=27), 
 param_grid = param_test4, scoring='accuracy',n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_train,Y_train)
gsearch4.best_params_, gsearch1.best_score_